# NICU Analytics Pipeline Runner

This notebook provides a simplified interface to run the modularized NICU analytics pipeline.

## Prerequisites

1. Set environment variables for Snowflake authentication:
   - `MY_SF_USER`: Your Snowflake username
   - `MY_SF_PKEY`: Your Snowflake private key (PEM format)

2. Ensure the NRS package is in your Python path

In [ ]:
# Import the NRS pipeline
import sys
import logging

# Add parent directory to path if needed
# sys.path.append('..')

from NRS import run_nicu_pipeline

## Option 1: Run with Auto-Calculated Birth Windows

This is the recommended approach. The pipeline will automatically calculate the appropriate birth windows and runout periods based on available data.

In [ ]:
# Run pipeline with automatic window calculation
run_nicu_pipeline(
    client_data='EMBLEM',
    auto_window=True,
    log_level=logging.INFO
)

## Option 2: Run with Manual Birth Windows

If you need to specify custom date ranges, you can provide them manually.

In [ ]:
from datetime import datetime

# Run pipeline with manual date ranges
run_nicu_pipeline(
    client_data='EMBLEM',
    auto_window=False,
    birth_window_start=datetime(2023, 1, 1),
    birth_window_end=datetime(2024, 12, 31),
    runout_end=datetime(2025, 3, 31),
    log_level=logging.INFO
)

## Advanced Usage: Step-by-Step Execution

For debugging or custom workflows, you can import and run individual pipeline components.

In [ ]:
from NRS import get_snowflake_session, ReferenceDataManager, TABLE_CONFIG
from NRS.membership import process_membership
from NRS.utils import calculate_birth_window, setup_logging

# Setup logging
setup_logging(level=logging.INFO)

# Initialize session
session = get_snowflake_session()

# Calculate birth windows
client = 'EMBLEM'
birth_start, birth_end, birth_mid, runout = calculate_birth_window(
    session, client, TABLE_CONFIG
)

print(f"Birth Window: {birth_start.date()} to {birth_end.date()}")
print(f"Runout End: {runout.date()}")

In [ ]:
# Process membership (example of individual step)
membership_df = process_membership(
    session,
    client,
    birth_start,
    birth_mid,
    birth_end,
    client,
    TABLE_CONFIG
)

print(f"Membership processed: {membership_df.count()} records")

## Configuration

View and modify pipeline configuration settings.

In [ ]:
from NRS import BUSINESS_RULES, CPT_CONFIG, REV_CODE_CONFIG

print("Business Rules:")
print(f"  Birth Window Months: {BUSINESS_RULES.birth_window_months}")
print(f"  Readmit Threshold Days: {BUSINESS_RULES.readmit_threshold_days}")
print(f"  High Cost Threshold: ${BUSINESS_RULES.high_cost_threshold:,}")

print("\nCPT Configuration:")
print(f"  Manageable CPTs: {CPT_CONFIG.manageable_cpts}")
print(f"  Critical Care CPTs: {CPT_CONFIG.critical_care_cpts}")

print("\nRevenue Code Configuration:")
print(f"  Room & Board Prefixes: {REV_CODE_CONFIG.room_board_prefixes}")
print(f"  NICU Level Range: {REV_CODE_CONFIG.nicu_level_min}-{REV_CODE_CONFIG.nicu_level_max}")

## Next Steps

After the pipeline completes:

1. **Membership Output**: `CSZNB_PRD_PS_PFA_DB.BASE.PS_MEMBERSHIP_{CLIENT}_TST`
2. **Newborns Output**: `CSZNB_PRD_PS_PFA_DB.BASE.PS_NEWBORNS_{CLIENT}_TST`

These tables contain the final analytical outputs ready for reporting and analysis.